In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

In [ ]:
prefix_path = "/Users/skoneru/workspace/interview_prep/ml_100_days/datasets/instacart-market-basket-analysis"

In [ ]:
products = pd.read_csv(Path(f"{prefix_path}/products.csv"))
print("\n Products Table")
print(products.head())

aisles = pd.read_csv(Path(f"{prefix_path}/aisles.csv"))
print("\n Aisles Table")
print(aisles.head())

departments = pd.read_csv(Path(f"{prefix_path}/departments.csv"))
print("\n Departments Table")
print(departments.head())



In [ ]:
orders_data = pd.read_csv(Path(f"{prefix_path}/orders.csv"))
print(orders_data.head())
print(orders_data.info())
print(orders_data.describe(include=[np.number]))
print(orders_data.describe(exclude=[np.number]))

order_products_prior = pd.read_csv(Path(f"{prefix_path}/order_products__prior.csv"))
print(order_products_prior.head())
print(order_products_prior.info())
print(order_products_prior.describe(include=[np.number]))


order_products_train = pd.read_csv(Path(f"{prefix_path}/order_products__train.csv"))
print(order_products_train.head())
print(order_products_train.info())
print(order_products_train.describe(include=[np.number]))


In [ ]:
full_products = products.merge(aisles, how="left", on="aisle_id")
full_products = full_products.merge(departments, how="left", on="department_id")
full_products

In [ ]:
full_order_products_prior = order_products_prior.merge(orders_data, how="left", on="order_id")
full_order_products_train = order_products_train.merge(orders_data, how="left", on="order_id")

full_order_products_prior = full_order_products_prior.merge(full_products, how="left", on="product_id")
full_order_products_train = full_order_products_train.merge(full_products, how="left", on="product_id")

full_order_products_prior.info()
full_order_products_train.info()

# Compute User Stats

In [ ]:
user_orders = orders_data.groupby("user_id").size().reset_index(name="total_orders")
print(user_orders.head())

user_products_features = full_order_products_prior.groupby(["user_id", "product_id"]).size().reset_index(name="total_bought")
user_products_features = user_products_features.sort_values(by=["user_id", "product_id"])
print(user_products_features.head())

# Compute average order size per user 
order_quantity = full_order_products_prior.groupby(["user_id", "order_id", "product_id"]).size().reset_index(name="order_quantity")
print(order_quantity.head())
order_quantity = order_quantity.groupby(["user_id", "product_id"])["order_quantity"].mean().reset_index(name="avg_order_quantity")
print(order_quantity.head())

# compute number of times a product was bought by a user on last 5 orders
last_5_orders = orders_data[orders_data["order_number"] > 5]
last_order_products_prior = last_5_orders.merge(order_products_prior, how="left", on="order_id")
print(last_order_products_prior.info())

last_n_user_products_features = last_order_products_prior.groupby(["user_id", "product_id"]).size().reset_index(name="total_bought_last_5_orders")
last_n_user_products_features = last_n_user_products_features.sort_values(by=["user_id", "product_id"])
print(last_n_user_products_features.head())



In [ ]:
# reorder ration of a product by a user
reorder_ratio = full_order_products_prior.groupby(["user_id","product_id"])["reordered"].mean().reset_index(name="reorder_ratio")
reorder_ratio.sort_values(by="reorder_ratio", ascending=False, inplace=True)
print(reorder_ratio.head())

In [ ]:

prior_features = user_products_features.merge( user_orders, how="left", on=["user_id",  "product_id"])
prior_features = prior_features.merge(order_quantity, how="left", on=["user_id", "product_id"])
prior_features = prior_features.merge(last_n_user_products_features, how="left", on=["user_id", "product_id"])
prior_features = prior_features.merge(reorder_ratio, how="left", on=["user_id", "product_id"])


prior_features